Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [fire.ca.gov](https://www.fire.ca.gov/what-we-do/fire-resource-assessment-program/gis-mapping-and-data-analytics).
<hr>

# Rhode Island Wildfire Compare

The code below is used to clean the raw information collected by the [California Department of Forestry and Fire Protection](https://www.fire.ca.gov/what-we-do/fire-resource-assessment-program/gis-mapping-and-data-analytics) and [Canadian Wildland Fire Information System](https://cwfis.cfs.nrcan.gc.ca/). The code gathers the largest fires, shifts and reporjects them over Rhode Island to easily compare the size with the smallest state. The polygons are then simplified using topojson to make for a faster loading web experience and the details of the fires are saved to json.

**The Canandian fires were added recently and only reflect active fires for 2023 not a history of older fires like the California dataset*  
**The Canandian fires were limited to top 10, because the fires lack names in the dataset the uid was used*
<hr>


In [1]:
import requests
import pandas as pd
import geopandas as gpd
import topojson as tp

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

In [2]:
# functions to shift polygons to RI location
def shift_distance(centroid):
    if centroid.y > ri_y:
        return [abs(centroid.x - ri_x), -abs(centroid.y - ri_y)]   
        
    return [abs(centroid.x - ri_x), abs(centroid.y - ri_y)]  

def shift_polygon(geom):
    centroid = geom.centroid
    shift = shift_distance(centroid)
    
    gs = gpd.GeoSeries(geom)
    gs = gs.translate(shift[0], shift[1])

    return gs[0]

In [3]:
def simplify_geo(df):
    gdf = gpd.GeoSeries(df['geometry'])
    topo = tp.Topology(gdf, prequantize=False)
    gdf = topo.toposimplify(50).to_gdf()
    df['geometry'] = gdf['geometry'].values
    
    return df

def combine_overlap(df):
    df = df.explode(index_parts=False)
    df['geometry'] = df['geometry'].make_valid()
    df = df[df['geometry'].is_valid]
    df = df.dissolve(by='FIRE_NAME').reset_index()
    
    return df

def convert_crs(df):
    df['geometry'] = df['geometry'].to_crs(epsg=4326)
    
    return df

def sort_values(df):
    df = df.sort_values(by='GIS_ACRES', ascending=False)
    
    return df

In [4]:
# download active fire data from canada
url = 'https://cwfis.cfs.nrcan.gc.ca/downloads/hotspots'
shp_ext = ['dbf', 'prj', 'shp', 'shx']

for ext in shp_ext:
    r = requests.get(f'{url}/perimeters.{ext}')
    with open(f'./data/raw/canada/perimeters.{ext}', 'wb') as f:
        f.write(r.content)

In [5]:
# get center of RI to shift polygons
ri = gpd.read_file('./data/raw/State_Boundary_(1997).json', layer=0)
ri = ri[ri['geometry'] != None]
ri['state'] = 'ri'
ri = ri.dissolve(by='state')
ri = ri.to_crs("epsg:4087")

ri_x = ri.centroid[0].x
ri_y = ri.centroid[0].y

In [6]:
# load cali data clean & prep top 20
df = gpd.read_file('./data/raw/california/fire22_1.gdb', layer='firep22_1')
df = df.sort_values(by='Shape_Area', ascending=False).head(20)
df['geometry'] = df['geometry'].to_crs(epsg=4087)

df['FIRE_NAME'] = df['FIRE_NAME'].str.lower()
df['GIS_ACRES'] = df['GIS_ACRES'].astype(int)

df['geometry'] = df['geometry'].apply(shift_polygon)

cali = (df
    .pipe(simplify_geo)
    .pipe(combine_overlap)
    .pipe(convert_crs)
)

In [7]:
# load canada data clean & prep top 10
df = gpd.read_file('./data/raw/canada/perimeters.shp')
df['GIS_ACRES'] = (df['AREA'] * 2.471).astype(int)
df = df.sort_values(by='AREA', ascending=False).head(10)

# get canadian provinces
province = gpd.read_file('./data/raw/canada_province.zip')
province = province[['PRENAME', 'geometry']]
province = province.to_crs(df.crs)

# join province and drop duplicates
df = df.sjoin(province, how='left', predicate='intersects')
df = df.drop_duplicates(subset='UID')

df['geometry'] = df['geometry'].to_crs(epsg=4087)
df['geometry'] = df['geometry'].apply(shift_polygon)

# normalize names and types to match cali dataset
df = df.rename(columns={'UID': 'FIRE_NAME', 'FIRSTDATE': 'ALARM_DATE', 'LASTDATE': 'CONT_DATE'})
df['FIRE_NAME'] = df['FIRE_NAME'].astype(str)

ca = (df
    .pipe(simplify_geo)
    .pipe(combine_overlap)
    .pipe(convert_crs)
)

In [8]:
# loop over each cali fire name and save the file as geojson
df = cali.copy()
for name in df['FIRE_NAME'].unique():
    temp_df = df[df['FIRE_NAME'] == name]
    fname = name.strip().replace(' ', '-')
    temp_df.to_file(f'./data/clean/50/{fname}.geojson', driver='GeoJSON')
    
# format dates
df['ALARM_DATE'] = pd.to_datetime(df['ALARM_DATE']).dt.strftime('%#m/%d/%Y')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE']).dt.strftime('%#m/%d/%Y')

df['ALARM_DATE'] = df['ALARM_DATE'].astype(str)
df['CONT_DATE'] = df['CONT_DATE'].astype(str)
df['CONT_DATE'] = df['CONT_DATE'].fillna('(N/A)')

# rename year col re-sort
df = df.sort_values(by='GIS_ACRES', ascending=False)

cols = ['FIRE_NAME', 'ALARM_DATE', 'CONT_DATE', 'GIS_ACRES']
df[cols].to_json('./data/clean/california.json', orient='records')

In [9]:
# loop over each canada fire id and save the file as geojson
df = ca.copy()
for name in df['FIRE_NAME'].unique():
    temp_df = df[df['FIRE_NAME'] == name]
    fname = name.strip().replace(' ', '-')
    temp_df.to_file(f'./data/clean/50/{fname}.geojson', driver='GeoJSON')
    
# format dates
df['ALARM_DATE'] = pd.to_datetime(df['ALARM_DATE']).dt.strftime('%#m/%d/%Y')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE']).dt.strftime('%#m/%d/%Y')

df['ALARM_DATE'] = df['ALARM_DATE'].astype(str)
df['CONT_DATE'] = df['CONT_DATE'].astype(str)
df['CONT_DATE'] = df['CONT_DATE'].fillna('(N/A)')

# re sort
df = df.sort_values(by='GIS_ACRES', ascending=False)

# add a new name for dropdown (province -> rank)
df['PRENAME'] = df['PRENAME'].str.replace('North West', 'NW')
df['region_id'] = df.groupby('PRENAME').cumcount() + 1
df['DROP_NAME'] = df['PRENAME'] + " - " + df['region_id'].astype(str)

cols = ['FIRE_NAME', 'DROP_NAME', 'ALARM_DATE', 'CONT_DATE', 'GIS_ACRES']
df[cols].to_json('./data/clean/canada.json', orient='records')